<a href="https://colab.research.google.com/github/jayarnim/project-Comparative_Analysis_of_Changes_in_Influence_of_Market_Demand_Determinants/blob/main/code/0_1_df_parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/2023-2_MachineLearning/Project/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/2023-2_MachineLearning/Project


In [2]:
from common import sector, data, variable
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv(data.PATH_DF_MERGE)

df = df.dropna()

all_col = set(df.columns)
drop_col = {'업종'} | set(df.loc[:,'추정매출_비율_주중주말_주중':'추정매출_비율_연령대별_60이상'].columns) | set(df.loc[:,'추정매출_점포수':'점포현황_점포수_프랜차이즈'].columns)
sum_col = set(df.loc[:,'추정매출_금액_주중주말_주중':'추정매출_건수_연령대별_60이상'].columns) | {'추정매출_금액', '추정매출_건수'}
maintain_col = all_col - (drop_col | sum_col)

col_sequence = [col for col in df.columns if (col in sum_col) or (col in maintain_col)]

In [8]:
for i in tqdm(iterable = range(len(sector.CODELIST)), desc = "업종"):
    result_list = []

    parsed_df = df[df['업종'].isin(sector.CODELIST[i])]

    selected_df = parsed_df[maintain_col | sum_col]

    unique_dict = dict(selected_df[['년도','분기','상권유형','상권']].value_counts())
    unique_list = list(unique_dict.keys())

    for j in range(len(unique_list)):
        conditions = (
            selected_df['년도'] == unique_list[j][0]
            ) & (
                selected_df['분기'] == unique_list[j][1]
                ) & (
                    selected_df['상권유형'] == unique_list[j][2]
                    ) & (
                        selected_df['상권'] == unique_list[j][3]
                        )

        sum_dict = dict(selected_df[conditions][sum_col].sum())
        maintain_dict = dict(selected_df[conditions][maintain_col].min())

        result = sum_dict
        result.update(maintain_dict)
        result_list.append(result)

    result_df = pd.DataFrame(result_list)
    result_df = result_df[col_sequence]

    result_df.to_csv(f"data/{sector.NAMELIST[i]}.csv", index = False)

업종: 100%|██████████| 12/12 [1:01:42<00:00, 308.51s/it]
